In [1]:
import gzip
import numpy as np
import csv, os
import pandas as pd
from os import listdir
from os.path import isfile, join
import gzip
import sys

In [2]:
data = 'GSE75140'
inpath = "./new dataset/"  # Because your files are directly in the current folder
outpath = "./output/"  # Create a local folder to save outputs
try:
    os.makedirs(outpath)
except FileExistsError:
    print("Output folder already exists.")
    

Output folder already exists.


In [3]:
#meta data
meta = pd.read_csv(inpath + data + '_meta.csv')


In [4]:
aux = pd.read_csv(inpath+data + '_sample.csv')
raw_data_file = open(inpath + 'GSE75140_hOrg.fetal.master.data.frame.txt')
raw_data_lines = raw_data_file.readlines()
raw_data_file.close()

In [5]:
#construct the experiment list from the raw data
Exp_Name_list = []
for idx_raw, raw_data in enumerate(raw_data_lines):
    if idx_raw > 0: #first line is the header
        raw_data = raw_data.split()
        Exp_Name_list.append(eval(raw_data[0]))

In [6]:
aux['Title'].tolist()


['whOrg1_B12_hOrg33d_c2',
 'whOrg1_B1_hOrg33d_c2',
 'whOrg1_B6_hOrg33d_c2',
 'whOrg1_B7_hOrg33d_c2',
 'whOrg1_C10_hOrg33d_c2',
 'whOrg1_C11_hOrg33d_c2',
 'whOrg1_C8_hOrg33d_c2',
 'whOrg1_D2_hOrg33d_c2',
 'whOrg1_F1_hOrg33d_c2',
 'whOrg1_F6_hOrg33d_c2',
 'whOrg1_G10_hOrg33d_c1',
 'whOrg1_G11_hOrg33d_c1',
 'whOrg1_G12_hOrg33d_c1',
 'whOrg1_G1_hOrg33d_c1',
 'whOrg1_G2_hOrg33d_c1',
 'whOrg1_G3_hOrg33d_c1',
 'whOrg1_G4_hOrg33d_c1',
 'whOrg1_G5_hOrg33d_c1',
 'whOrg1_G6_hOrg33d_c1',
 'whOrg1_G6_hOrg33d_c2']

In [7]:
#Construct the sample list from aux file
Sample_Name_list = []
for exp in Exp_Name_list:
    found = False
    for idx_title, title in enumerate(aux['Title']):
        title = title.split('_')
        title = '_'.join(title[1:])
        if exp == title:
            found = True
            break
    if found == False and exp == 'F5_fetal_12wpc_c1':
        Sample_Name_list.append('GSM1957793')
    elif found == False:
        print('Cannot find experiment name in aux file', exp)
        break
    else:
        Sample_Name_list.append(aux['Accession'][idx_title])

for sample in meta['GEO_Accession (exp)']:
    if sample not in Sample_Name_list:
        print(sample)

Cannot find experiment name in aux file A10_hOrg_ESC_reg1_53d
GSM1957048
GSM1957049
GSM1957050
GSM1957051
GSM1957052
GSM1957053
GSM1957054
GSM1957055
GSM1957056
GSM1957057
GSM1957058
GSM1957059
GSM1957060
GSM1957061
GSM1957062
GSM1957063
GSM1957064
GSM1957065
GSM1957066
GSM1957067
GSM1957068
GSM1957069
GSM1957070
GSM1957071
GSM1957072
GSM1957073
GSM1957074
GSM1957075
GSM1957076
GSM1957077
GSM1957078
GSM1957079
GSM1957080
GSM1957081
GSM1957082
GSM1957083
GSM1957084
GSM1957085
GSM1957086
GSM1957087
GSM1957088
GSM1957089
GSM1957090
GSM1957091
GSM1957092
GSM1957093
GSM1957094
GSM1957095
GSM1957096
GSM1957097
GSM1957098
GSM1957099
GSM1957100
GSM1957101
GSM1957102
GSM1957103
GSM1957104
GSM1957105
GSM1957106
GSM1957107
GSM1957108
GSM1957109
GSM1957110
GSM1957111
GSM1957112
GSM1957113
GSM1957114
GSM1957115
GSM1957116
GSM1957117
GSM1957118
GSM1957119
GSM1957120
GSM1957121
GSM1957122
GSM1957123
GSM1957124
GSM1957125
GSM1957126
GSM1957127
GSM1957128
GSM1957129
GSM1957130
GSM1957131
GSM1957132
GSM

In [8]:
# Construct the sample list from aux file
Cell_type_list = []  #construct the cell list from STAGE in meta
for sample in Sample_Name_list:
    found = False
    for idx_geo, geo in enumerate(meta['GEO_Accession (exp)']):
        if geo == sample:
            found = True
            break
    if found == False:
        print('Cannot find geo in meta data')
    if sample == 'GSM1957673' and meta['STAGE'][idx_geo] == '-':
        print('Correcting cell type for', sample)
        Cell_type_list.append('12 weeks post-conception')
    else:
        Cell_type_list.append(meta['STAGE'][idx_geo])


In [9]:
no_dup = 0
for idx_s, sample in enumerate(Sample_Name_list):
    Exp_Name_list[idx_s] = Exp_Name_list[idx_s].split()[0]
    found = False
    for idx, geo in enumerate(meta['GEO_Accession (exp)']):
        if sample == geo:
            if found == False:
                found = True
                index = idx
            else:
                no_dup += 1
                if meta['major_cell_type'][idx] != meta['major_cell_type'][index]:
                    print('cell type of duplicate does not match!')
                    break
    if found == False:
        print('Sample name not found in meta data')

In [10]:
unique = ['12 weeks post-conception',
            '13 weeks post-conception',
            '33 days',
            '35 days',
            '37 days',
            '41 days',
            '53 days',
            '58 days',
            '65 days']

N = len(Sample_Name_list)
print('Number of cells:', N)
print('Cell types:', len(unique))

Number of cells: 0
Cell types: 9


In [11]:
#construct the gene list
gene_list = raw_data_lines[0]   #the first row contains the gene 
gene_list = gene_list.split()[1:-1]
for idx_gene, gene in enumerate(gene_list):
    gene_list[idx_gene] = eval(gene).upper()

print('Number of Genes:', len(gene_list))
        
N = len(raw_data_lines) - 1

MATRIX = np.zeros([len(gene_list), N])

Number of Genes: 18927


In [12]:

for idx_raw, raw_data in enumerate(raw_data_lines):
    idx_col = idx_raw-1
    if idx_raw > 0:
        raw_data = raw_data.split()[1:-1]
        data_col = np.zeros([len(raw_data)])
        if len(raw_data) != len(gene_list):
            print('Number of entries does not match')
            break
        for idx_gene in range(len(raw_data)):
            data_col[idx_gene] = float(raw_data[idx_gene])
        MATRIX[:, idx_col] = data_col


In [13]:
#################
#################
#Write data for full data
#cell_type_unique = list(set(list(cell_type_full_list)))  #used if the cell_type makes sense
print('Writing the Full Data>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
cell_type_unique = unique


Writing the Full Data>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


In [14]:
#write the dictionary
cell_type_dict = {i: cell_type_unique[i] for i in range(len(cell_type_unique)) }  #dict to map index to cell type
cell_type_dict_rev = {cell_type_unique[i]:i for i in range(len(cell_type_unique))}  #reverse dict from cell type to index
file = open(outpath + data + '_full_labeldict.txt', 'w')   #write a dictionary
file.writelines(str(cell_type_dict) + ' \n')
file.writelines(str(cell_type_dict_rev) + ' \n')
file.close()
cell_count =  {cell_type_unique[i]:0 for i in range(len(cell_type_unique))}
Sample_Name_list = Sample_Name_list
Cell_label_list = []
for idx in range(len(Cell_type_list)):  #iterate over all the samples
    cell_type = Cell_type_list[idx]     #get the current cell type
    try:   #see if the cell type exist in the dictionary
        cell_label = cell_type_dict_rev[cell_type]
        cell_count[cell_type] += 1
        Cell_label_list.append(cell_label)
        found = True
    except:
        found = False    #if it is not found, print message
        print(Sample_Name_list[idx], cell_type, 'Wrong cell type')

print('Cell Count')

Cell Count


In [15]:
for k in cell_count.keys():
    print('%s:'%k, cell_count[k])

with open(outpath + data + '_full_labels.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Sample Name', 'Cell type', 'Label'])
    for idx in range(len(Cell_type_list)):
        writer.writerow([Sample_Name_list[idx], Cell_type_list[idx] , Cell_label_list[idx]])
print(len(Sample_Name_list), 'sample names')
print(len(Cell_type_list), 'cell types')
print(len(Cell_label_list), 'cell labels')
                
with open(outpath + data + '_full_gene.csv', "w", newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(['Index', 'Gene'])
    for idx, gene in enumerate(gene_list):
        writer.writerow([idx, gene])

12 weeks post-conception: 0
13 weeks post-conception: 0
33 days: 0
35 days: 0
37 days: 0
41 days: 0
53 days: 0
58 days: 0
65 days: 0
0 sample names
0 cell types
0 cell labels


In [16]:
with open(outpath + data + '_full_data.csv', "w", newline = '') as csv_file: #output file for data
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(['Row', 'Col', 'Val'])
    for idx_col in range(len(Sample_Name_list)):
        for idx_row in range(len(gene_list)):
            if MATRIX[idx_row, idx_col] > 0:
                writer.writerow([idx_row, idx_col, MATRIX[idx_row, idx_col]])
                    
with open(outpath + data + '_full_X.csv', "w", newline = '') as csv_file: #output file for data
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow( [None] + Sample_Name_list)
    for idx_row in range(len(gene_list)):
        row = list(MATRIX[idx_row, :])
        writer.writerow([gene_list[idx_row] ] + row)
        